# Atividade: CNNs para Classificação

Neste notebook, iremos preparar nosso próprio dataset e treinar um modelo de classificação de imagens.

## Preparando os dados

Os dados desta atividade serão baixados da internet. Utilizaremos para isso buscadores comuns. Em seguida, dividiremos em treinamento e validação.

In [ ]:
!pip install icrawler

In [ ]:
import os
import shutil
import random
from icrawler.builtin import GoogleImageCrawler, BingImageCrawler

### Adquirindo as Imagens

Utilizaremos o iCrawler para baixar imagens em buscadores através de termos especificados. Defina sua lista de classes.

In [ ]:
def download_images(keyword, folder, n_total=100):
    os.makedirs(folder, exist_ok=True)
    downloaded = len(os.listdir(folder))
    remaining = n_total - downloaded

    while downloaded < n_total:
        crawler = GoogleImageCrawler(storage={'root_dir': folder})
        crawler.crawl(keyword=keyword, max_num=remaining, file_idx_offset=downloaded)
        downloaded = len(os.listdir(folder))
        remaining = n_total - downloaded
        print(f"Downloaded {downloaded}/{n_total}")

    print("Download complete!")

In [ ]:
# Definindo as duas classes e seus respectivos termos de busca
search_terms = {
    # Chave (Nome da Pasta): Valor (Palavra-Chave para a Busca)
    "Goku": "Goku Dragon Ball z alta qualidade",
    "Vegeta": "Vegeta Dragon Ball z alta qualidade",
}


base_folder = "data/anime_dbz"

for label, term in search_terms.items():
    print(f"Iniciando download para a classe: {label}...")

    # A variável 'label' agora define o nome da pasta
    download_images(term, f"{base_folder}/{label}", n_total=100)

print("\nTodos os downloads foram concluídos.")

Iniciando download para a classe: Goku...


ERROR:downloader:Exception caught when downloading file https://e7.png, error: HTTPSConnectionPool(host='e7.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b8c380e4c50>: Failed to resolve 'e7.png' ([Errno -2] Name or service not known)")), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://e7.png, error: HTTPSConnectionPool(host='e7.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b8c380e5190>: Failed to resolve 'e7.png' ([Errno -2] Name or service not known)")), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://e7.png, error: HTTPSConnectionPool(host='e7.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b8c380e5700>: Failed to resolve 'e7.png' ([Errno -2] Name or serv

Downloaded 66/100


ERROR:downloader:Exception caught when downloading file https://e7.png, error: HTTPSConnectionPool(host='e7.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b8c2a9d5a60>: Failed to resolve 'e7.png' ([Errno -2] Name or service not known)")), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://e7.png, error: HTTPSConnectionPool(host='e7.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b8c2a9d7440>: Failed to resolve 'e7.png' ([Errno -2] Name or service not known)")), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://e7.png, error: HTTPSConnectionPool(host='e7.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b8c2a9d78f0>: Failed to resolve 'e7.png' ([Errno -2] Name or serv

Downloaded 100/100
Download complete!
Iniciando download para a classe: Vegeta...


ERROR:downloader:Response status code 404, file https://static.wikia.nocookie.net/nerddragon/images/f/f5/Vegeta_Saga_Saiyajin_%28Forma_Base%29.png
ERROR:downloader:Exception caught when downloading file https://www.kindpng, error: HTTPSConnectionPool(host='www.kindpng', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b8c29f172c0>: Failed to resolve 'www.kindpng' ([Errno -2] Name or service not known)")), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://www.kindpng, error: HTTPSConnectionPool(host='www.kindpng', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b8c29f30530>: Failed to resolve 'www.kindpng' ([Errno -2] Name or service not known)")), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://www.kindpng, error: HTTPSConnectionPool(host='www.kindpng', por

Downloaded 68/100


ERROR:downloader:Response status code 404, file https://static.wikia.nocookie.net/nerddragon/images/f/f5/Vegeta_Saga_Saiyajin_%28Forma_Base%29.png
ERROR:downloader:Exception caught when downloading file https://www.kindpng, error: HTTPSConnectionPool(host='www.kindpng', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b8c385c21e0>: Failed to resolve 'www.kindpng' ([Errno -2] Name or service not known)")), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://www.kindpng, error: HTTPSConnectionPool(host='www.kindpng', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b8c385bb650>: Failed to resolve 'www.kindpng' ([Errno -2] Name or service not known)")), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://www.kindpng, error: HTTPSConnectionPool(host='www.kindpng', por

Downloaded 100/100
Download complete!

Todos os downloads foram concluídos.


### Treinamento e Validação

Dividiremos as imagens baixadas nas pastas `train` e `val`. Defina uma porcentagem.

In [ ]:
def split_train_val(root_dir, train_ratio=0.8, seed=42):
    random.seed(seed)

    train_dir = root_dir + "_split/train"
    val_dir = root_dir + "_split/val"

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    for class_name in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        images = [os.path.join(class_path, f) for f in os.listdir(class_path)]
        images = [f for f in images if os.path.isfile(f)]
        random.shuffle(images)

        n_train = int(len(images) * train_ratio)

        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        for img in images[:n_train]:
            shutil.copy(img, os.path.join(train_class_dir, os.path.basename(img)))
        for img in images[n_train:]:
            shutil.copy(img, os.path.join(val_class_dir, os.path.basename(img)))

        print(f"{class_name}: {n_train} train, {len(images)-n_train} val")

## Dataset

Implemente um Dataset PyTorch que carregue as imagens baixadas com suas respectivas classes. Aplique data augmentation e carregue em batches.

In [ ]:
import os
import random
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# --- 1. Definição da Classe Dataset Personalizada (COM DEBUG) ---
class AnimeDataset(Dataset):
    # ... (restante do código igual)

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Mapeamento de nome de pasta (classe) para índice numérico
        self.class_to_idx = {
            'Goku': 0,
            'Vegeta': 1
        }

        print(f"\n[DEBUG DATASET] Iniciando busca em: {self.root_dir}")

        # Preenche as listas de caminhos e rótulos
        for class_name, idx in self.class_to_idx.items():
            class_path = os.path.join(root_dir, class_name)

            if os.path.isdir(class_path):
                # Tenta listar os arquivos APENAS se o diretório existir
                try:
                    filenames = os.listdir(class_path)

                    # Filtra apenas arquivos (exclui pastas ocultas, etc.)
                    valid_files = [f for f in filenames if os.path.isfile(os.path.join(class_path, f))]

                    if len(valid_files) == 0:
                         print(f"[DEBUG DATASET] A pasta '{class_name}' está VAZIA. Caminho: {class_path}")
                         continue # Pula para a próxima classe

                    for filename in valid_files:
                        self.image_paths.append(os.path.join(class_path, filename))
                        self.labels.append(idx)

                    print(f"[DEBUG DATASET] Classe '{class_name}' adicionada com {len(valid_files)} imagens.")

                except Exception as e:
                    print(f"[DEBUG DATASET] ERRO de permissão/leitura em {class_path}. Erro: {e}")

            else:
                print(f"[DEBUG DATASET] Pasta de classe NÃO ENCONTRADA para '{class_name}'. Esperado em: {class_path}")

    def __len__(self):
        return len(self.image_paths)


    def __getitem__(self, idx):
        # Implementação de tratamento de erros com try-except
        try:
            # 1. Carregar a Imagem
            img_path = self.image_paths[idx]
            image = Image.open(img_path).convert('RGB')
            label = self.labels[idx]

            # 2. Aplicar Transformações
            if self.transform:
                image = self.transform(image)

            return image, label

        except Exception as e:
            # Em caso de erro (ex: arquivo corrompido, erro de leitura)
            print(f"Erro ao carregar imagem em {self.image_paths[idx]}. Erro: {e}")
            # Retorna um elemento aleatório para não quebrar o batching
            # Isso é uma solução simples; em produção, você pularia este item.
            # Aqui, tentamos uma solução para continuar o treinamento:

            # Escolhe um novo índice aleatório (que não seja o atual)
            new_idx = random.randint(0, len(self) - 1)
            if new_idx == idx:
                new_idx = (new_idx + 1) % len(self)

            return self.__getitem__(new_idx)


# --- 2. Definição das Transformações e Data Augmentation ---
def get_transforms(image_size=224):
    """Define as transformações para treino (com augmentation) e validação."""

    # Transformações para o conjunto de TREINO (com Data Augmentation)
    train_transforms = transforms.Compose([
        # Data Augmentation:
        transforms.Resize((image_size, image_size)), # Redimensiona para um tamanho padrão
        transforms.RandomRotation(15),             # Rotação aleatória de até 15 graus
        transforms.RandomHorizontalFlip(p=0.5),    # Flip horizontal aleatório
        transforms.ColorJitter(brightness=0.1, contrast=0.1), # Variações de cor/brilho

        # Conversão e Normalização (Essencial para pré-treinamento - ImageNet)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Transformações para o conjunto de VALIDAÇÃO/TESTE (SEM Data Augmentation)
    # Apenas redimensiona, converte e normaliza
    val_transforms = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    return train_transforms, val_transforms





In [ ]:
# --- 3. Função de Implementação e Carregamento (DataLoader) ---
def create_dataloaders(root_dir, batch_size=32, image_size=224):
    """Cria e retorna os DataLoaders de Treino e Validação."""

    # 🚨 Modificação: A função create_dataloaders deve receber o caminho base do SPLIT.
    # Ex: /content/data/anime_dbz_split



    if root_dir.endswith('/'):
        root_dir = root_dir[:-1] # Remove barra final, se houver

    # Assume que a pasta de treino/validação está em {root_dir}/train e {root_dir}/val
    train_dir = os.path.join(root_dir, "train")
    val_dir = os.path.join(root_dir, "val")

    # --- NOVO: Verificação do ponto de partida ---
    print(f"\n[DEBUG LOADER] Verificando diretórios esperados:")
    print(f"[DEBUG LOADER] Treino: {train_dir}")
    print(f"[DEBUG LOADER] Validação: {val_dir}")

    if not os.path.isdir(train_dir) or not os.path.isdir(val_dir):
        print("ERRO CRÍTICO: Pastas 'train' e/ou 'val' não foram encontradas. O caminho base do split está incorreto.")
        print("Verifique se o seu 'root_dir' na chamada é a pasta que contém 'train' e 'val'.")
        return None, None
    # ---------------------------------------------

    train_transforms, val_transforms = get_transforms(image_size)

    try:
        # Cria os Datasets (aqui o erro num_samples=0 será detectado pelos prints internos)
        train_dataset = AnimeDataset(root_dir=train_dir, transform=train_transforms)
        val_dataset = AnimeDataset(root_dir=val_dir, transform=val_transforms)

        # O código a seguir só será executado se len(train_dataset) e len(val_dataset) > 0

        if len(train_dataset) == 0 or len(val_dataset) == 0:
             # Este raise agora será mais específico com a informação de debug
             raise ValueError("Dataset vazio. Verifique os prints de debug acima para saber qual pasta falhou.")

        # ... (Criação dos DataLoaders)

        print(f"Datasets carregados: Treino ({len(train_dataset)} imagens), Validação ({len(val_dataset)} imagens)")

        return train_dataloader, val_dataloader

    except Exception as e:
        # ... (Tratamento de erro existente)
        return None, None

In [ ]:


# 1. Executar a Divisão do Dataset
# Esta chamada CRIA a pasta de saída: /content/data/anime_dbz_split
print("--- Passo 1: Executando a Divisão (SPLIT) dos dados ---")
split_train_val(
    root_dir='/content/data/anime_dbz',  # Pasta de entrada (com Goku/ e Vegeta/)
    train_ratio=0.8,
    seed=42
)
print("Divisão concluída. Pasta /content/data/anime_dbz_split criada.")


# 2. Criar os DataLoaders a partir do diretório CORRETO (o que foi recém-criado)
print("\n--- Passo 2: Criando DataLoaders a partir do diretório SPLIT ---")
if __name__ == '__main__':
    TRAIN_LOADER, VAL_LOADER = create_dataloaders(
        root_dir="/content/data/anime_dbz_split", # Pasta que agora existe
        batch_size=32
    )

    if TRAIN_LOADER and VAL_LOADER:
        # Teste de iteração (só rodará se o tamanho do dataset for > 0)
        print("\nSucesso! Testando um batch de treino:")
        for images, labels in TRAIN_LOADER:
            print(f"Shape do batch de imagens (Batch Size, Channels, Height, Width): {images.shape}")
            print(f"Labels do batch: {labels}")
            break

--- Passo 1: Executando a Divisão (SPLIT) dos dados ---
Goku: 80 train, 20 val
Vegeta: 80 train, 20 val
Divisão concluída. Pasta /content/data/anime_dbz_split criada.

--- Passo 2: Criando DataLoaders a partir do diretório SPLIT ---

[DEBUG LOADER] Verificando diretórios esperados:
[DEBUG LOADER] Treino: /content/data/anime_dbz_split/train
[DEBUG LOADER] Validação: /content/data/anime_dbz_split/val

[DEBUG DATASET] Iniciando busca em: /content/data/anime_dbz_split/train
[DEBUG DATASET] Classe 'Goku' adicionada com 80 imagens.
[DEBUG DATASET] Classe 'Vegeta' adicionada com 80 imagens.

[DEBUG DATASET] Iniciando busca em: /content/data/anime_dbz_split/val
[DEBUG DATASET] Classe 'Goku' adicionada com 20 imagens.
[DEBUG DATASET] Classe 'Vegeta' adicionada com 20 imagens.
Datasets carregados: Treino (160 imagens), Validação (40 imagens)


In [ ]:
!ls /content/data/anime_dbz


Goku  Vegeta


## Definição do Modelo

Defina aqui o modelo que será utilizado, sendo implementação própria ou um modelo pré-treinado.

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

def setup_model(num_classes=2, feature_extract=True):
    """
    Define e inicializa o modelo ResNet18 pré-treinado.

    Args:
        num_classes (int): O número de classes de saída (2 para Goku e Vegeta).
        feature_extract (bool): Se True, congela os parâmetros do corpo do modelo.
    """

    # 1. Carregar o modelo ResNet18 pré-treinado no ImageNet
    model_ft = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    print("Modelo base ResNet18 carregado com pesos pré-treinados.")


    if feature_extract:
        for param in model_ft.parameters():
            param.requires_grad = False



    num_ftrs = model_ft.fc.in_features

    # Substituir a camada 'fc' por uma nova camada linear
    model_ft.fc = nn.Linear(num_ftrs, num_classes)

    print(f"Camada final (fc) modificada para {num_classes} classes.")

    return model_ft

if __name__ == '__main__':
    NUM_CLASSES = 2

    # 1. Inicializar o Modelo
    model = setup_model(num_classes=NUM_CLASSES, feature_extract=True)

    # 2. Mover o modelo para a GPU, se disponível
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # 3. Testar o Modelo (Passar um tensor de dados pelo modelo)
    # Criamos um tensor simulando um batch de 32 imagens (cor: 3, tamanho: 224x224)
    dummy_input = torch.randn(32, 3, 224, 224).to(device)

    try:
        output = model(dummy_input)
        print(f"\nTeste de output do modelo realizado com sucesso.")
        print(f"Shape de saída (Batch Size, Classes): {output.shape}") # Deve ser [32, 2]
    except Exception as e:
        print(f"Erro ao passar dados de teste pelo modelo: {e}")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 48.5MB/s]


Modelo base ResNet18 carregado com pesos pré-treinados.
Camada final (fc) modificada para 2 classes.

Teste de output do modelo realizado com sucesso.
Shape de saída (Batch Size, Classes): torch.Size([32, 2])


## Treinamento

Defina a função de custo e o otimizador do modelo. Em seguida, implemente o código de treinamento e treine-o. Ao final, exiba as curvas de treinamento e validação para a loss e a acurácia.

In [ ]:
import os
import time
import copy
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt

class AnimeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {'Goku': 0, 'Vegeta': 1}

        for class_name, idx in self.class_to_idx.items():
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                for filename in os.listdir(class_path):
                    fpath = os.path.join(class_path, filename)
                    if os.path.isfile(fpath):
                        self.image_paths.append(fpath)
                        self.labels.append(idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            img_path = self.image_paths[idx]
            image = Image.open(img_path).convert('RGB')
            label = self.labels[idx]
            if self.transform:
                image = self.transform(image)
            return image, label

        except Exception as e:
            new_idx = random.randint(0, len(self) - 1)
            return self.__getitem__(new_idx)

def get_transforms(image_size=224):
    train_transforms = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.RandomRotation(15),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    val_transforms = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return train_transforms, val_transforms

def create_dataloaders(root_dir, batch_size=32, image_size=224):
    train_dir = os.path.join(root_dir, "train")
    val_dir = os.path.join(root_dir, "val")
    train_transforms, val_transforms = get_transforms(image_size)

    try:
        train_dataset = AnimeDataset(root_dir=train_dir, transform=train_transforms)
        val_dataset = AnimeDataset(root_dir=val_dir, transform=val_transforms)

        if len(train_dataset) == 0 or len(val_dataset) == 0:
             raise ValueError("Dataset vazio.")

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

        return train_dataloader, val_dataloader

    except Exception as e:
        return None, None

def setup_model(num_classes=2, feature_extract=True):
    model_ft = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    if feature_extract:
        for param in model_ft.parameters():
            param.requires_grad = False

    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    return model_ft

def train_model(model, dataloaders, criterion, optimizer, num_epochs=15):
    since = time.time()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    return model, history

def plot_curves(history):
    epochs = range(1, len(history['train_loss']) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], 'bo-', label='Loss Treinamento')
    plt.plot(epochs, history['val_loss'], 'ro-', label='Loss Validação')
    plt.title('Curvas de Loss')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_acc'], 'bo-', label='Acurácia Treinamento')
    plt.plot(epochs, history['val_acc'], 'ro-', label='Acurácia Validação')
    plt.title('Curvas de Acurácia')
    plt.xlabel('Época')
    plt.ylabel('Acurácia')
    plt.legend()

    plt.show()

if __name__ == '__main__':
    DATA_DIR = "/content/data/anime_dbz_split"
    NUM_CLASSES = 2
    NUM_EPOCHS = 15
    LEARNING_RATE = 0.001

    train_loader, val_loader = create_dataloaders(DATA_DIR, batch_size=32)

    if train_loader is not None and val_loader is not None:
        dataloaders_dict = {'train': train_loader, 'val': val_loader}

        model_ft = setup_model(NUM_CLASSES, feature_extract=True)
        criterion = nn.CrossEntropyLoss()

        params_to_update = [p for p in model_ft.parameters() if p.requires_grad]
        optimizer_ft = optim.Adam(params_to_update, lr=LEARNING_RATE)

        _, history = train_model(
            model_ft,
            dataloaders_dict,
            criterion,
            optimizer_ft,
            num_epochs=NUM_EPOCHS
        )

        plot_curves(history)

## Inferência

Calcule algumas métricas como acurácia, matriz de confusão, etc. Em seguida, teste o modelo em novas imagens das classes correspondentes mas de outras fontes (outro buscador, fotos próprias, etc).

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from torchvision import transforms, models
import random
import time
import copy

# --- FUNÇÕES ESSENCIAIS DE ETAPAS ANTERIORES ---

class AnimeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {'Goku': 0, 'Vegeta': 1}

        for class_name, idx in self.class_to_idx.items():
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                for filename in os.listdir(class_path):
                    fpath = os.path.join(class_path, filename)
                    if os.path.isfile(fpath):
                        self.image_paths.append(fpath)
                        self.labels.append(idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            img_path = self.image_paths[idx]
            image = Image.open(img_path).convert('RGB')
            label = self.labels[idx]
            if self.transform:
                image = self.transform(image)
            return image, label

        except Exception as e:
            new_idx = random.randint(0, len(self) - 1)
            return self.__getitem__(new_idx)

def get_transforms(image_size=224):
    train_transforms = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.RandomRotation(15),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    val_transforms = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return train_transforms, val_transforms

def create_dataloaders(root_dir, batch_size=32, image_size=224):
    train_dir = os.path.join(root_dir, "train")
    val_dir = os.path.join(root_dir, "val")
    train_transforms, val_transforms = get_transforms(image_size)

    try:
        train_dataset = AnimeDataset(root_dir=train_dir, transform=train_transforms)
        val_dataset = AnimeDataset(root_dir=val_dir, transform=val_transforms)

        if len(train_dataset) == 0 or len(val_dataset) == 0:
             raise ValueError("Dataset vazio.")

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

        return train_dataloader, val_dataloader

    except Exception as e:
        return None, None

def setup_model(num_classes=2, feature_extract=True):
    model_ft = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    if feature_extract:
        for param in model_ft.parameters():
            param.requires_grad = False

    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    return model_ft

# --- FUNÇÕES DE INFERÊNCIA E AVALIAÇÃO ---

def evaluate_model(model, dataloader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.eval()
    criterion = nn.CrossEntropyLoss()

    running_loss = 0.0
    running_corrects = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    total_loss = running_loss / len(dataloader.dataset)
    total_acc = running_corrects.double() / len(dataloader.dataset)
    cm = confusion_matrix(all_labels, all_preds)

    return total_acc.item(), total_loss, cm

def plot_confusion_matrix(cm, class_names=['Goku', 'Vegeta']):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Matriz de Confusão')
    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.show()

def test_new_image(model, image_path, class_names=['Goku', 'Vegeta']):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.eval()

    _, val_transforms = get_transforms(image_size=224)

    try:
        image = Image.open(image_path).convert('RGB')
    except:
        return

    input_tensor = val_transforms(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.softmax(output, dim=1)[0]
        _, predicted_class_idx = torch.max(output, 1)

    predicted_label = class_names[predicted_class_idx.item()]
    confidence = probabilities[predicted_class_idx.item()].item() * 100

    plt.imshow(image)
    plt.title(f"PREDIÇÃO: {predicted_label} ({confidence:.2f}%)")
    plt.axis('off')
    plt.show()

# --- BLOCO PRINCIPAL DE EXECUÇÃO (INFERÊNCIA) ---

if __name__ == '__main__':

    DATA_DIR = "/content/data/anime_dbz_split"
    NUM_CLASSES = 2

    # 1. Carregar DataLoaders
    _, val_loader = create_dataloaders(DATA_DIR, batch_size=32)

    # 2. Configurar Modelo
    modelo_final = setup_model(NUM_CLASSES, feature_extract=True)

    # 3. Simular Carga de Pesos Treinados (SUBSTITUA PELA CARGA REAL DOS SEUS PESOS TREINADOS)
    # Exemplo: modelo_final.load_state_dict(torch.load('melhores_pesos.pt'))
    # Para fins de demonstração, o modelo inicializado será usado.

    print("--- INFERÊNCIA NO CONJUNTO DE VALIDAÇÃO ---")

    if val_loader:
        acc, loss, cm = evaluate_model(modelo_final, val_loader)

        print(f"Loss na Validação: {loss:.4f}")
        print(f"Acurácia na Validação: {acc:.4f} ({acc*100:.2f}%)")

        plot_confusion_matrix(cm)

    print("\n--- TESTE EM NOVAS IMAGENS (SIMULAÇÃO) ---")

    # 4. Configuração de Novas Imagens de Teste
    # SUBSTITUA ESTA LISTA COM OS CAMINHOS REAIS DE NOVAS IMAGENS BAIXADAS EXTERNAMENTE!
    imagens_teste_simuladas = [
        '/content/goku_ssj3_new.jpg',
        '/content/vegeta_blue_new.jpg',
    ]

    if not any(os.path.exists(p) for p in imagens_teste_simuladas):
         print("Nenhum caminho de imagem de teste externo válido fornecido. Teste ignorado.")
    else:
        for i, img_path in enumerate(imagens_teste_simuladas):
            if os.path.exists(img_path):
                test_new_image(modelo_final, img_path)